# Plot Monitors on top of AOD using `cartopy`

In my [last notebook][notebook-link], I overlayed state capitol locations on top of an AOD plot using `cartopy`. Now, I will try to do the same with air quality monitors. This will involve retrieving monitoring data in R using the `PWFSLSmoke` package, importing the data into Python, and plotting in on top of the AOD plot using color to represent AQI values.

[notebook-link]: add_known_locations_to_cartopy_plots_2019-04-05.ipynb

## Goal

Plot air quality monitors colored by AQI value on top of AOD data 

## Generate monitoring data

To generate a CSV containing the latitudes, longitudes and pm2.5 values for all airsis, WRCC, and EPA air quality monitors, I used the following R script, which can also be found in `write_monitor_csv.R`.

```R
library(PWFSLSmoke)
library(lubridate)

datetime <- "201903191512"

aod_datetime <- as_datetime(datetime,
                            format="%Y%m%d%H%M")

# Retrieve data for 3 days preceding the time that the AOD data was collected
# to apply nowcast algorithm
monitor <- monitor_load(aod_datetime - days(3), aod_datetime)

# apply nowcast algorithm and retrieve the hour associated with AOD data
monitor_data <-
  monitor_nowcast(monitor) %>%
  monitor_extractData() %>%
  dplyr::filter(datetime == floor_date(aod_datetime, unit = "hours")) %>%
  dplyr::select(-datetime) %>%
  as.numeric()

tbl <- tibble(
  latitude = monitor$meta$latitude,
  longitude = monitor$meta$longitude,
  value = monitor_data
)

readr::write_csv(tbl, sprintf("monitors_%s.csv", datetime))
```

## Read in air quality monitor data

In [1]:
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
from pyproj import Proj
import xarray
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import os
import sys

In [5]:
df_monitor = pd.read_csv("../../local_data/monitors_2019031915.csv")

print("Shape: {}".format(df_monitor.shape))


Shape: (925, 3)
